In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import glob
import xml.etree.ElementTree as ET
import fnmatch
import os,sys
import random
from PIL import Image

- Load Detections file for both open and closed state of object class
- Create folders inside the detections directory for storing images and label
- Generate labels

In [23]:
# Root folder for Detections
det_root = 'det_val'
object_name = 'laptop'
images_root = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/JPEGImages/'

# creating file name for open and closed state
open_file = det_root + '/' + det_root + '_' + object_name + '_' + 'open' + object_name.title() + '.txt'
close_file = det_root + '/' + det_root + '_' + object_name + '_' + 'closed' + object_name.title() + '.txt'
print open_file
print close_file
header = ['img_id', 'confidence', 'x1', 'y1', 'x2', 'y2']

#load detections of open state
open_data = pd.read_csv(open_file, names=header,  delimiter=' ')
# Assigning Label to Open state
open_data['label'] = 0

#load detection of closed state
close_data = pd.read_csv(close_file, names=header, delimiter=' ')
# Assigning Label to Closed state
close_data['label'] = 1

# concat open and close into single frame
data = pd.concat([open_data, close_data], ignore_index=True)

# generate new name for each patch
data['new_id'] = data['img_id'].astype(str) + '_' + data.index.astype(str)
data = data.sort_values(by=['confidence'], ascending=False)
#sanity check
#print open_data.head(5)
print open_data.shape, close_data.shape, data.shape
print data.head(5)
print data.tail(5)

image_dir = det_root + '/images/' + object_name + '/'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
        
labels_dir = det_root + '/labels/'
if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
    
label_fileName = labels_dir + object_name + '_' + det_root[4:] + '.txt'
print label_fileName

det_val/det_val_laptop_openLaptop.txt
det_val/det_val_laptop_closedLaptop.txt
(6005, 7) (2786, 7) (8791, 8)
                         img_id  confidence     x1     y1     x2     y2  \
6849  COCO_val2014_000000509811         1.0  233.6  256.5  318.8  315.5   
3551  COCO_val2014_000000574424         1.0   91.0  303.8  341.2  480.0   
8510  COCO_val2014_000000233638         1.0  507.9  173.9  630.5  357.0   
4903  COCO_val2014_000000437893         1.0  176.7  257.8  471.5  426.0   
1502  COCO_val2014_000000170436         1.0  136.8  195.4  298.0  375.0   

      label                          new_id  
6849      1  COCO_val2014_000000509811_6849  
3551      0  COCO_val2014_000000574424_3551  
8510      1  COCO_val2014_000000233638_8510  
4903      0  COCO_val2014_000000437893_4903  
1502      0  COCO_val2014_000000170436_1502  
                         img_id  confidence     x1     y1     x2     y2  \
5099  COCO_val2014_000000424842        0.05  320.1  430.5  632.5  482.4   
8612  COCO_val2

- Loop through each row and load image
- extract the patch
- assign new name to patch and save it in images directory
- concat name and label into single string and append it in list
- perform sanity check number elements in list and data must be same
- Dump list to file to generate labels 

In [19]:
#sample = data.iloc[0:10]
#print sample
labels_list = []
for index, row in data.iterrows():
    
    # load, crop and save image
    img = Image.open(images_root + row['img_id'] + '.jpg')
    new_img = img.crop((row['x1'], row['y1'], row['x2'], row['y2']))
    new_img.save(image_dir + row['new_id'] + '.jpg')
    
    # generate labels
    lbl = row['new_id'] + '.jpg' + ' ' + str(row['label']) + '\n'
    labels_list.append(lbl)
#print labels_list

#write labels into file
with open(label_fileName, 'w+') as f:
    for val in labels_list:
        f.write(val)
        
print len(labels_list)

17599
